# Model Development
____

This notebook goes into creating supervised model that are used to predict the price of cars. Two models are derrived from this and are saved to be used for future inference.

## Funtion Definitions
____

In [ ]:
import joblib
import numpy as np
import os
import optuna
import pandas as pd

from datetime import datetime

from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures

from xgboost import XGBRegressor

In [ ]:
def get_dataTypes_and_missingValues(df):
    info = pd.DataFrame()
    info['data_types'] =  df.dtypes
    info['unique_values'] = df.nunique()
    info['missing_values'] = df.isna().sum()
    return info


In [ ]:
def extract_and_split_dataset(path, drop_cols, split=True):
    
    df = pd.read_csv(path)
    df.drop_duplicates(inplace=True)
    df.drop(drop_cols, axis=1, inplace=True)

    if split:
        X = df.drop("Price", axis=1)
        y = df["Price"]
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
        train_df = pd.concat([X_train, y_train], axis=1)
        val_df = pd.concat([X_val, y_val], axis=1)
        return train_df, val_df

    return df

In [ ]:
def extract_numeric_features(X, columns_to_extract):
    X_copy = X.copy()
    for col in columns_to_extract:
        X_copy[col] = pd.to_numeric(X_copy[col].str.split(' ').str[0], downcast='float', errors='coerce')
    return X_copy[columns_to_extract]

def preprocess_levy_and_fillna(X):

    X_copy = X.copy()
    X_copy["Levy"].replace("-", None, inplace=True)

    X_copy['Levy'] = pd.to_numeric(X_copy['Levy'], errors='coerce')
    mean_levy_by_year = X_copy.groupby('Prod. year')['Levy'].mean()
    mean_levy_by_year.fillna(0, inplace=True)
    
    for year in X_copy['Prod. year'].unique():
        mask = (X_copy['Prod. year'] == year) & X_copy['Levy'].isnull()
        X_copy.loc[mask, 'Levy'] = mean_levy_by_year[year]
    
    X_copy['Levy'] = X_copy['Levy'].astype(int)
    
    return X_copy

def preprocessing_pipeline(numeric_features_to_extract, ordinal_features, categorical_features, other_features):
    """This function performs the pre-processing steps for the features and retuns an numeric representation of all the features which is used to train the model"""
    
    
    column_transformer = make_column_transformer(
            (FunctionTransformer(preprocess_levy_and_fillna, validate=False), ['Prod. year', 'Levy']),
            (FunctionTransformer(extract_numeric_features, kw_args={'columns_to_extract': numeric_features_to_extract}), numeric_features_to_extract),
            (OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int), ordinal_features),
            (OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False), categorical_features),
            ("drop", other_features)
    )

    return column_transformer

In [ ]:
def get_trained_model(X_train, y_train, model_algorithm, random_state=124, **model_kwargs):
    if 'random_state' in model_algorithm().get_params():
        model_kwargs['random_state'] = random_state

    model = model_algorithm(**model_kwargs)

    numeric_features = X_train.select_dtypes(np.number).columns.to_list()
    extract_num_feats = ['Mileage', 'Engine volume']
    ord_feats = ['Manufacturer', 'Model', 'Fuel type']
    cat_feats = ['Leather interior', 'Gear box type', 'Category']
    others = list(set(X_train.columns) - set(numeric_features + extract_num_feats +
                                             ['Levy'] + cat_feats + ord_feats))
    ct = preprocessing_pipeline(numeric_features_to_extract=extract_num_feats,
                                 ordinal_features=ord_feats,
                                 categorical_features=cat_feats, other_features=others)

    pipeline = make_pipeline(ct, StandardScaler(), model)
    pipeline.fit(X_train, y_train.values.ravel())

    return pipeline

## Pre Processing
____

Changes from the last notebook:

1. **Label Encoding:**
   - The columns 'Manufacturer' and 'Model' will be label encoded, converting them to numerical representations.

2. **Irrelevant Columns:**
   - Some columns have been identified as irrelevant and will be dropped. Ideally the decision to drop columns would involve domain expertise or subject matter experts (SMEs).

| Dropped Column | Reason                                      |
| -------------- | ------------------------------------------- |
| Cylinders      | Information is contained in Engine volume, Manufacturer, Model, and Category. |
| Drive Wheels   | Similar information is captured elsewhere.   |
| Doors          | Considered irrelevant.                       |
| Wheel          | Location-specific information.               |
| Color          | Deemed irrelevant for the analysis.          |
| Airbags        | Considered irrelevant for predicting price. |

These changes aim to streamline the dataset by removing redundant or less impactful features.

In [ ]:
# Define the DataFrame-based features
dir = os.path.join('.', 'data', 'raw', 'train.csv')
drop_cols = ['ID', 'Cylinders', 'Drive wheels', 'Doors', 'Wheel', 'Color', 'Airbags']
train_df, val_df = extract_and_split_dataset(dir, drop_cols)
get_dataTypes_and_missingValues(train_df)

,data_types,unique_values,missing_values
Levy,object,526,0
Manufacturer,object,64,0
Model,object,1392,0
Prod. year,int64,50,0
Category,object,11,0
Leather interior,object,2,0
Fuel type,object,7,0
Engine volume,object,103,0
Mileage,object,6486,0
Gear box type,object,4,0


In [ ]:
numeric_features = train_df.select_dtypes(np.number).drop('Price', axis=1).columns.to_list()
extract_num_feats = ['Mileage', 'Engine volume']
ord_feats = ['Manufacturer', 'Model', 'Fuel type']
cat_feats = ['Leather interior','Gear box type', 'Category']
others = list(set(train_df.columns) - set(numeric_features + extract_num_feats + ['Levy','Price'] + cat_feats + ord_feats))
ct = preprocessing_pipeline(numeric_features_to_extract=extract_num_feats,ordinal_features=ord_feats, 
                            categorical_features=cat_feats, other_features=others)

# remove prices less than $1000
train_df = train_df[train_df['Price'] >= 1000]
X_train = train_df.drop('Price', axis=1).reset_index(drop=True)
y_train = train_df[['Price']].reset_index(drop=True)

In [ ]:
X_val = val_df.drop('Price', axis=1).reset_index(drop=True)
y_val = val_df[['Price']].reset_index(drop=True)

## Linear Models
____

In [ ]:
scoring = {
    'neg_rmse': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
    'neg_mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

pipe = make_pipeline(ct, StandardScaler(), LinearRegression())
scores = cross_validate(pipe, X_train, y_train, scoring=scoring, return_train_score=True)
cv_df = pd.DataFrame(scores)
cv_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,0.170814,0.055032,-29500.578227,-256333.935412,-15500.503915,-17236.019883
1,0.137396,0.055028,-30542.405235,-256334.131923,-14798.064705,-16036.635123
2,0.131754,0.054711,-28643.536914,-256354.815042,-14489.660853,-16568.787232
3,0.133644,0.068409,-31700.611445,-256300.424642,-14097.338817,-15969.625100
4,0.130267,0.057206,-513850.918281,-18373.273789,-20015.595340,-10110.062342


In [ ]:
poly = PolynomialFeatures(degree=1)

pipe = make_pipeline(ct, StandardScaler(), poly, LinearRegression())
scores = cross_validate(pipe, X_train, y_train, scoring=scoring, return_train_score=True)
cv_df = pd.DataFrame(scores)
cv_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,0.127763,0.054871,-29444.796131,-256333.468373,-15456.903065,-17209.919351
1,0.124056,0.056146,-30507.494618,-256334.278342,-14778.701923,-16030.863264
2,0.120629,0.053845,-28657.116339,-256355.269208,-14480.270217,-16571.223276
3,0.134053,0.053781,-31655.347565,-256298.327786,-14106.439712,-16044.255635
4,0.138274,0.130213,-513847.683840,-18375.854170,-20031.247687,-10138.069408


In [ ]:
poly = PolynomialFeatures(degree=2)

pipe = make_pipeline(ct, StandardScaler(), poly, LinearRegression())
scores = cross_validate(pipe, X_train, y_train, scoring=scoring, return_train_score=True)
cv_df = pd.DataFrame(scores)
cv_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,0.650070,0.061416,-7.418861e+10,-249882.186484,-1.596250e+09,-19219.697749
1,0.466598,0.129184,-1.659277e+12,-253632.291454,-4.578407e+10,-16669.550168
2,0.489255,0.062922,-8.806342e+05,-252041.672386,-4.269042e+04,-17623.493115
3,0.442200,0.120054,-4.029299e+05,-253020.681497,-2.699216e+04,-16653.150913
4,0.456578,0.060704,-7.427665e+11,-15745.533557,-1.538676e+10,-8857.767338


Above it seems like increasing the degree of freedom begins to overfit the model.

In [ ]:
pipe = make_pipeline(ct, StandardScaler(), Ridge(alpha=0.001))
scores = cross_validate(pipe, X_train, y_train, scoring=scoring, return_train_score=True)
cv_df = pd.DataFrame(scores)
cv_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,0.190814,0.049670,-29439.746391,-256333.399409,-15452.107931,-17209.033399
1,0.103515,0.050985,-30510.526030,-256333.665228,-14782.266151,-15995.077318
2,0.168856,0.050008,-28633.293675,-256354.859544,-14486.912158,-16551.543957
3,0.104054,0.050121,-31643.274171,-256297.215706,-14127.277136,-16061.604706
4,0.170362,0.049042,-513851.545448,-18373.351619,-20017.436432,-10112.846938


The above showcases a feature of linear regression related to regularization. Lasso or ElasticNet are recommended approaches when dealing with datasets that have a substantial number of features, as is the case here. It's important to note that Lasso, while effective, may exhibit a longer convergence time compared to ElasticNet. 

In [ ]:
pipe = make_pipeline(ct, StandardScaler(), ElasticNet(alpha=0.001))
scores = cross_validate(pipe, X_train, y_train, cv=2, return_train_score=True, scoring=scoring)
cv_df = pd.DataFrame(scores)
cv_df

/home/ec2-user/.car-prices-prediction/lib64/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.419e+14, tolerance: 6.937e+10
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/.car-prices-prediction/lib64/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.797e+11, tolerance: 2.686e+08
  model = cd_fast.enet_coordinate_descent(


,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,0.319381,0.076683,-42649.060491,-323110.245312,-18967.359772,-22267.826145
1,0.258117,0.075128,-325275.926814,-17293.267563,-14081.723794,-10070.759702


## Ensemble Techniques
____


The linear models exhibit sub-optimal performance,thet ensemble techniques might yield better results.
* Utilizing a Random Forest Regressor for bagging.
* 
Employing an XGBoost Regressor for boosting.

--> Random Forest Regressor

In [ ]:
rf_pipe = make_pipeline(ct, StandardScaler(), RandomForestRegressor())
rf_scores = cross_validate(rf_pipe, X_train, y_train.values.ravel(), scoring=scoring, return_train_score=True)
rf_df = pd.DataFrame(rf_scores)
rf_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,5.009693,0.132792,-343672.667973,-125943.870059,-17065.023470,-4259.494996
1,5.124063,0.135222,-16703.607478,-127674.795976,-5856.140884,-4606.214561
2,4.824773,0.133104,-10332.761630,-108754.223953,-5179.019694,-4613.119585
3,4.797619,0.134260,-20543.212985,-100762.600004,-5490.103241,-4749.378293
4,4.584707,0.131690,-513664.199830,-5194.255601,-15048.202796,-2042.589494


Root Mean Squared Error (RMSE) is more sensitive to outliers compared to Mean Absolute Error (MAE), as evidenced by the broader range of values.

<!-- Default scoring is 'neg_mean_squared_error' for Mean Squared Error (MSE) -->--> XGB Regressor

In [ ]:
xgb_pipe = make_pipeline(ct, StandardScaler(), XGBRegressor())
xgb_scores = cross_validate(xgb_pipe, X_train, y_train, scoring=scoring, return_train_score=True)
xgb_df = pd.DataFrame(xgb_scores)
xgb_df

,fit_time,score_time,test_neg_rmse,train_neg_rmse,test_neg_mae,train_neg_mae
0,1.847337,0.061041,-509996.129713,-5484.398557,-15749.847274,-3646.019321
1,1.536711,0.062022,-11854.437565,-5438.273834,-5989.564691,-3596.556270
2,1.489095,0.131222,-11194.303958,-5523.512809,-6008.727843,-3681.244221
3,1.552229,0.092416,-16897.649489,-5902.424604,-5895.344839,-3868.452781
4,1.544082,0.116141,-513647.674401,-5244.705145,-15628.569695,-3416.997986


Based on these values the Random Forest Regressor seems to perform better than the XGB Regressor

## Hyper-parameter Tuning
____

#### --> Random Forest Regressor

In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 1, 32)
    min_samples_split = trial.suggest_float('min_samples_split', 0.001, 0.5, log=True)
    min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.001, 0.5, log=True)
    max_features = trial.suggest_float('max_features', 0.1, 1.0)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model_kwargs = {'n_estimators': n_estimators,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'max_features': max_features,
                    'bootstrap': bootstrap,
                    'n_jobs':-1
                    }

    model = get_trained_model(X_train, y_train, RandomForestRegressor, **model_kwargs)
    
    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    trial.report(mae, step=trial.number)
    if trial.should_prune():
        raise optuna.TrialPruned()    

    return mae

rf_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner())
rf_study.optimize(objective, n_trials=50)

[I 2023-12-27 00:51:25,845] A new study created in memory with name: no-name-0fc2da14-aa35-4bb5-8632-2a6bf74ebdd2
[I 2023-12-27 00:51:27,878] Trial 0 finished with value: 11994.321184127182 and parameters: {'n_estimators': 241, 'max_depth': 32, 'min_samples_split': 0.002621027568944447, 'min_samples_leaf': 0.06906094000938462, 'max_features': 0.525436010262137, 'bootstrap': True}. Best is trial 0 with value: 11994.321184127182.
[I 2023-12-27 00:51:28,460] Trial 1 finished with value: 10306.236900896882 and parameters: {'n_estimators': 48, 'max_depth': 31, 'min_samples_split': 0.0042638078426972216, 'min_samples_leaf': 0.002118996252061989, 'max_features': 0.16866457400703438, 'bootstrap': True}. Best is trial 1 with value: 10306.236900896882.
[I 2023-12-27 00:51:30,422] Trial 2 pruned. 
[I 2023-12-27 00:51:31,839] Trial 3 pruned. 
[I 2023-12-27 00:51:35,733] Trial 4 pruned. 
[I 2023-12-27 00:51:36,156] Trial 5 pruned. 
[I 2023-12-27 00:51:46,089] Trial 6 pruned. 
[I 2023-12-27 00:51:46

In [ ]:
print("Best Trial:")
rf_best_trial = rf_study.best_trial
print("Value: ", rf_best_trial.value)
print("Params: ")
for key, value in rf_best_trial.params.items():
    print(f"    {key}: {value}")

Best Trial:
Value:  8323.113058519546
Params: 
    n_estimators: 97
    max_depth: 22
    min_samples_split: 0.0023146536832538587
    min_samples_leaf: 0.001002134328961941
    max_features: 0.9414935783510276
    bootstrap: False


The trail above takes roughly 15 minutes going through 50 trials.

#### -->XGB Regressor

In [ ]:
def objective(trial):

    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    min_child_weight = trial.suggest_float('min_child_weight', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0.0, 1.0)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)

    model_kwargs = {
                    'colsample_bytree': colsample_bytree,
                    'min_child_weight': min_child_weight,
                    'learning_rate': learning_rate,
                    'gamma': gamma,
                    'max_depth': max_depth,
                    'n_estimators': n_estimators,
                    'subsample': subsample,
                    'reg_alpha': reg_alpha,
                    'reg_lambda': reg_lambda,
                    }

    model = get_trained_model(X_train, y_train, XGBRegressor, **model_kwargs)

    y_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    trial.report(mae, step=trial.number)
    if trial.should_prune():
        raise optuna.TrialPruned()   
    
    return mae

xgb_study = optuna.create_study(direction='minimize', pruner=optuna.pruners.SuccessiveHalvingPruner())
xgb_study.optimize(objective, n_trials=50)

[I 2023-12-27 01:03:39,874] A new study created in memory with name: no-name-7608d87f-953a-4f8c-a6af-91cac0e6c404
[I 2023-12-27 01:03:55,425] Trial 0 finished with value: 16423.08538298525 and parameters: {'colsample_bytree': 0.5159720641992551, 'min_child_weight': 6.5345431720916825, 'learning_rate': 0.025034979032681065, 'gamma': 0.36085100328987973, 'max_depth': 9, 'n_estimators': 891, 'subsample': 0.9575718682495609, 'reg_alpha': 0.759695659812304, 'reg_lambda': 0.017177658357771408}. Best is trial 0 with value: 16423.08538298525.
[I 2023-12-27 01:03:57,883] Trial 1 finished with value: 21547.743247895134 and parameters: {'colsample_bytree': 0.643528101849511, 'min_child_weight': 8.411085664282417, 'learning_rate': 0.23842487432007137, 'gamma': 0.9229116951102323, 'max_depth': 7, 'n_estimators': 128, 'subsample': 0.5464639402097278, 'reg_alpha': 0.4813666619761755, 'reg_lambda': 0.3351180798715955}. Best is trial 0 with value: 16423.08538298525.
[I 2023-12-27 01:04:10,828] Trial 2 

In [ ]:
print("Best Trial:")
xgb_best_trial = xgb_study.best_trial
print("Value: ", xgb_best_trial.value)
print("Params: ")
for key, value in xgb_best_trial.params.items():
    print(f"    {key}: {value}")

Best Trial:
Value:  9045.705606913598
Params: 
    colsample_bytree: 0.9976711378207281
    min_child_weight: 3.157431071502319
    learning_rate: 0.01094008867924328
    gamma: 0.7032856276348437
    max_depth: 10
    n_estimators: 351
    subsample: 0.9118162108383395
    reg_alpha: 0.43473034704658026
    reg_lambda: 0.2305809419908718


The Random Forest Regressor yielded a superior best trial compared to the XGBoost Regressor. However, it's noteworthy that when I conducted this experiment locally, as opposed to on AWS Cloud9 where it's currently running, the XGBoost Regressor exhibited faster runtime and superior results.

## Saving the Models

Retraining the ensemble models with the best hyper parameters and saving these models and the transformer in a file so that they can be reused.

In [ ]:
def save_model(model, filename_prefix):
    """
    Save a machine learning model along with the current date and version.

    Parameters:
    - model: The machine learning model to be saved.
    - filename_prefix: A prefix for the filename indicating the type of model.

    Returns:
    - filename: The filename of the saved model.
    """
    date = datetime.now().strftime("%Y-%m-%d")
    version = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f'./models/{filename_prefix}_v{version}.joblib'

    # save file
    if not os.path.exists('./models'):
        os.makedirs('./models')
    joblib.dump({'model': model, 'date': date, 'version': version}, filename)

    return filename

In [ ]:
def train_and_save_model(X_train, y_train, model_class, **params):
    """
    Train a machine learning model, save it along with the current date and version, and return the filename.

    Parameters:
    - X_train: The training features.
    - y_train: The training labels.
    - model_class: The class of the machine learning model to be trained.
    - filename_prefix: A prefix for the filename indicating the type of model.
    - **params: Additional parameters to be passed to the model during initialization.

    Returns:
    - filename: The filename of the saved model.
    """
    
    model = get_trained_model(X_train, y_train, model_class, **params)
    filename_prefix = model_class.__name__

    filename = save_model(model, filename_prefix)

    return filename

In [ ]:
rf_best_trial.params, xgb_best_trial.params

({'n_estimators': 97,
  'max_depth': 22,
  'min_samples_split': 0.0023146536832538587,
  'min_samples_leaf': 0.001002134328961941,
  'max_features': 0.9414935783510276,
  'bootstrap': False},
 {'colsample_bytree': 0.9976711378207281,
  'min_child_weight': 3.157431071502319,
  'learning_rate': 0.01094008867924328,
  'gamma': 0.7032856276348437,
  'max_depth': 10,
  'n_estimators': 351,
  'subsample': 0.9118162108383395,
  'reg_alpha': 0.43473034704658026,
  'reg_lambda': 0.2305809419908718})

In [ ]:
rf_filename = train_and_save_model(X_train, y_train, RandomForestRegressor, **rf_best_trial.params)
xgb_filename = train_and_save_model(X_train, y_train, XGBRegressor, **xgb_best_trial.params)
ct_filename = save_model(ct, 'columntransformer')

## Future Improvements

- Currently, there is no baseline model in production; the current model represents the best available option.
  
- To improve model performance, further optimizations and feature engineering can be explored to create more optimal models.

- The next step involves saving these models for reuse. In future notebooks, a model registry will be implemented to store various models, facilitating the selection of optimal models.

- Deployment plans are underway to make these models accessible for predicting car prices. Once deployed, users will be able to input data, and the model will calculate the corresponding car price.
